In [1]:
import cv2
import tempfile
import imutils
import numpy as np
from PIL import Image
from matplotlib.pyplot import imshow

%matplotlib inline

Make sure you gave installed opencv(ver 3.x) package

pip install opencv-python imutils

In [2]:
image = cv2.imread('test2.jpg', 1)
#image = image.astype('uint8')

# Recomendtaions for improving quality of OCR by Tesseracy

[soruce](https://tesseract-ocr.github.io/tessdoc/ImproveQuality.html)

- [ ] Inverting images
- [x] Rescaling tp 300 dpi
- [ ] Binarisation [open CV](https://docs.opencv.org/master/d7/d4d/tutorial_py_thresholding.html)
- [ ] Noise Removal
- [ ] Dilation and Erosion
- [ ] Rotation / Deskewing
- [ ] Scanning border Removal
- [ ] Missing borders

---
## Page segmentation method

By default Tesseract expects a page of text when it segments an image. If you’re just seeking to OCR a small region, try a different segmentation mode, using the --psm argument. Note that adding a white border to text which is too tightly cropped may also help, see issue 398.

To see a complete list of supported page segmentation modes, use tesseract -h. Here’s the list as of 3.21:

  0    Orientation and script detection (OSD) only.
  1    Automatic page segmentation with OSD.
  2    Automatic page segmentation, but no OSD, or OCR.
  3    Fully automatic page segmentation, but no OSD. (Default)
  4    Assume a single column of text of variable sizes.
  5    Assume a single uniform block of vertically aligned text.
  6    Assume a single uniform block of text.
  7    Treat the image as a single text line.
  8    Treat the image as a single word.
  9    Treat the image as a single word in a circle.
 10    Treat the image as a single character.
 11    Sparse text. Find as much text as possible in no particular order.
 12    Sparse text with OSD.
 13    Raw line. Treat the image as a single text line,
			bypassing hacks that are Tesseract-specific.


In [15]:
def set_image_dpi(file_path):
    im = Image.open(file_path)
    length_x, width_y = im.size
    factor = min(1, float(1024.0 / length_x))
    size = int(factor * length_x), int(factor * width_y)
    im_resized = im.resize(size, Image.ANTIALIAS)
    temp_file = tempfile.NamedTemporaryFile(delete=False,   suffix='.png')
    temp_filename = temp_file.name
    im_resized.save(temp_filename, dpi=(300, 300))
    return temp_filename

In [17]:
fn = set_image_dpi("test2.jpg")

In [24]:
# this piece of code works if the text, say, located on the white that lies on the black table
# otherwise screenCnt remains None
# on the input the colord picture is given

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#gray = cv2.GaussianBlur(gray, (5, 5), 0)
edged = cv2.Canny(gray, 10, 50)
cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if imutils.is_cv2() else cnts[1]
cnts = sorted(cnts, key=cv2.contourArea, reverse=True)[:5]
print(len(cnts))

screenCnt = None
for c in cnts:
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)
    if len(approx) == 4:
        screenCnt = approx
        print('break')
        break

#im1 = Image.open(r'test_cv.jpg', mode='r')
#im1 = Image.fromarray(edged)
im1 = Image.fromarray(cv2.drawContours(image, [screenCnt], -1, (0, 255, 0), 2))
im1.save('test_cv.jpg')

5


error: OpenCV(3.4.2) /io/opencv/modules/imgproc/src/drawing.cpp:2612: error: (-215:Assertion failed) reader.ptr != __null in function 'cvDrawContours'


In [5]:
# one batch of funcitons to rotate the image with any rotation

def order_points(pts):
    # initialzie a list of coordinates that will be ordered
    # such that the first entry in the list is the top-left,
    # the second entry is the top-right, the third is the
    # bottom-right, and the fourth is the bottom-left
    rect = np.zeros((4, 2), dtype="float32")

    # the top-left point will have the smallest sum, whereas
    # the bottom-right point will have the largest sum
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]

    # now, compute the difference between the points, the
    # top-right point will have the smallest difference,
    # whereas the bottom-left will have the largest difference
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]

    # return the ordered coordinates
    return rect


def four_point_transform(image, pts):
    # obtain a consistent order of the points and unpack them
    # individually
    rect = order_points(pts)
    (tl, tr, br, bl) = rect

    # compute the width of the new image, which will be the
    # maximum distance between bottom-right and bottom-left
    # x-coordiates or the top-right and top-left x-coordinates
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

    # compute the height of the new image, which will be the
    # maximum distance between the top-right and bottom-right
    # y-coordinates or the top-left and bottom-left y-coordinates
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))

    # now that we have the dimensions of the new image, construct
    # the set of destination points to obtain a "birds eye view",
    # (i.e. top-down view) of the image, again specifying points
    # in the top-left, top-right, bottom-right, and bottom-left
    # order
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype="float32")

    # compute the perspective transform matrix and then apply it
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    return warped

In [6]:
# second approach to rotate an image
# doesn't work correctly. Possibly, it works with angels multiples 45 
def rotate(img):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray_img = cv2.bitwise_not(gray_img)
    coordinates = np.column_stack(np.where(gray_img > 0))
    ang = cv2.minAreaRect(coordinates)[-1]
    print(ang)
    if ang < -45: 
        ang = -(90 + ang)
    else:
        ang = -ang
    height, width = img.shape[:2]

    center_img = (width / 2, height / 2)

    rotationMatrix = cv2.getRotationMatrix2D(center_img, ang, 1.0)

    rotated_img = cv2.warpAffine(img, rotationMatrix, (width, height), borderMode = cv2.BORDER_REFLECT)
    return rotated_img

In [9]:
rotated = rotate(image)

-90.0


In [10]:
im1 = Image.open(r'test_cv.jpg', mode='r')
im1 = Image.fromarray(rotated)
im1.save('test_cv.jpg')

FileNotFoundError: [Errno 2] No such file or directory: 'test_cv.jpg'

In [7]:
def remove_noise_and_smooth(img):
    # input should be grayscale image
    filtered = cv2.adaptiveThreshold(img.astype(np.uint8), 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 9, 41)
    kernel = np.ones((1, 1), np.uint8)
    opening = cv2.morphologyEx(filtered, cv2.MORPH_OPEN, kernel)
    closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)
    # bluring (it can be as you wish. See the doc)
    img = cv2.GaussianBlur(img,(3,3),0)
    or_image = cv2.bitwise_or(img, closing)
    return or_image

In [8]:
# rotate image
ratio = 1
pts = np.array(screenCnt.reshape(4, 2) * ratio)
warped = four_point_transform(image, pts)



AttributeError: 'NoneType' object has no attribute 'reshape'

In [ ]:
im1 = Image.open(r'test_cv.jpg', mode='r')
im1 = Image.fromarray(warped)
im1.save('test_cv.jpg')

In [20]:
removed_noise = remove_noise_and_smooth(cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY))

In [34]:
# sharp the image
kernel_sharpening = np.array([[-1,-1,-1], 
                              [-1, 8,-1],
                              [-1,-1,-1]])

sharpened = cv2.filter2D(smooth, -1, kernel_sharpening)

In [26]:
# another sharp (or not)
blur = cv2.GaussianBlur(removed_noise,(3,3),0)
smooth = cv2.addWeighted(blur,1.5,removed_noise,-0.5,0)

In [35]:
im1 = Image.open(r'test_cv.jpg', mode='r')
im1 = Image.fromarray(sharpened)
im1.save('test_cv.jpg')

In [ ]:
cv2.imshow('Original Image', remove_noise_and_smooth(cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY))) 
cv2.waitKey(0)

# Sources

[\[1\]](https://answers.opencv.org/question/121205/how-to-refine-the-edges-of-an-image/)

[\[2\]](https://medium.com/cashify-engineering/improve-accuracy-of-ocr-using-image-preprocessing-8df29ec3a033)

# Script

[source](https://www.danvk.org/2015/01/07/finding-blocks-of-text-in-an-image-using-python-opencv-and-numpy.html)

In [1]:
import glob
import os
import random
import sys
import random
import math
import json
from collections import defaultdict

import cv2
from PIL import Image, ImageDraw
import numpy as np
from scipy.ndimage.filters import rank_filter

In [2]:
def dilate(ary, N, iterations): 
    """Dilate using an NxN '+' sign shape. ary is np.uint8."""
    kernel = np.zeros((N,N), dtype=np.uint8)
    
    kernel[int((N-1)/2) ,:] = 1
    dilated_image = cv2.dilate(ary / 255, kernel, iterations=iterations)

    kernel = np.zeros((N,N), dtype=np.uint8)
    kernel[:,int((N-1)/2)] = 1
    dilated_image = cv2.dilate(dilated_image, kernel, iterations=iterations)
    return dilated_image


def props_for_contours(contours, ary):
    """Calculate bounding box & the number of set pixels for each contour."""
    c_info = []
    for c in contours:
        x,y,w,h = cv2.boundingRect(c)
        c_im = np.zeros(ary.shape)
        cv2.drawContours(c_im, [c], 0, 255, -1)
        c_info.append({
            'x1': x,
            'y1': y,
            'x2': x + w - 1,
            'y2': y + h - 1,
            'sum': np.sum(ary * (c_im > 0))/255
        })
    return c_info


def union_crops(crop1, crop2):
    """Union two (x1, y1, x2, y2) rects."""
    x11, y11, x21, y21 = crop1
    x12, y12, x22, y22 = crop2
    return min(x11, x12), min(y11, y12), max(x21, x22), max(y21, y22)


def intersect_crops(crop1, crop2):
    x11, y11, x21, y21 = crop1
    x12, y12, x22, y22 = crop2
    return max(x11, x12), max(y11, y12), min(x21, x22), min(y21, y22)


def crop_area(crop):
    x1, y1, x2, y2 = crop
    return max(0, x2 - x1) * max(0, y2 - y1)


def find_border_components(contours, ary):
    borders = []
    area = ary.shape[0] * ary.shape[1]
    for i, c in enumerate(contours):
        x,y,w,h = cv2.boundingRect(c)
        if w * h > 0.5 * area:
            borders.append((i, x, y, x + w - 1, y + h - 1))
    return borders


def angle_from_right(deg):
    return min(deg % 90, 90 - (deg % 90))


def remove_border(contour, ary):
    """Remove everything outside a border contour."""
    # Use a rotated rectangle (should be a good approximation of a border).
    # If it's far from a right angle, it's probably two sides of a border and
    # we should use the bounding box instead.
    c_im = np.zeros(ary.shape)
    r = cv2.minAreaRect(contour)
    degs = r[2]
    if angle_from_right(degs) <= 10.0:
        box = cv2.cv.BoxPoints(r)
        box = np.int0(box)
        cv2.drawContours(c_im, [box], 0, 255, -1)
        cv2.drawContours(c_im, [box], 0, 0, 4)
    else:
        x1, y1, x2, y2 = cv2.boundingRect(contour)
        cv2.rectangle(c_im, (x1, y1), (x2, y2), 255, -1)
        cv2.rectangle(c_im, (x1, y1), (x2, y2), 0, 4)

    return np.minimum(c_im, ary)


def find_components(edges, max_components=16):
    """Dilate the image until there are just a few connected components.
    Returns contours for these components."""
    # Perform increasingly aggressive dilation until there are just a few
    # connected components.
    count = 21
    dilation = 5
    n = 1
    print('image dtype ',edges.dtype)
    while count > 16:
        n += 1
        dilated_image = dilate(edges, N=3, iterations=n)#.astype(np.uint8)
        dilated_image = np.uint8(dilated_image)
        _, contours, hierarchy = cv2.findContours(dilated_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        count = len(contours)
    #print dilation
    #Image.fromarray(edges).show()
    #Image.fromarray(255 * dilated_image).show()
    return contours


def find_optimal_components_subset(contours, edges):
    """Find a crop which strikes a good balance of coverage/compactness.
    Returns an (x1, y1, x2, y2) tuple.
    """
    c_info = props_for_contours(contours, edges)
    c_info.sort(key=lambda x: -x['sum'])
    total = np.sum(edges) / 255
    area = edges.shape[0] * edges.shape[1]

    c = c_info[0]
    del c_info[0]
    this_crop = c['x1'], c['y1'], c['x2'], c['y2']
    crop = this_crop
    covered_sum = c['sum']

    while covered_sum < total:
        changed = False
        recall = 1.0 * covered_sum / total
        prec = 1 - 1.0 * crop_area(crop) / area
        f1 = 2 * (prec * recall / (prec + recall))
        #print '----'
        for i, c in enumerate(c_info):
            this_crop = c['x1'], c['y1'], c['x2'], c['y2']
            new_crop = union_crops(crop, this_crop)
            new_sum = covered_sum + c['sum']
            new_recall = 1.0 * new_sum / total
            new_prec = 1 - 1.0 * crop_area(new_crop) / area
            new_f1 = 2 * new_prec * new_recall / (new_prec + new_recall)

            # Add this crop if it improves f1 score,
            # _or_ it adds 25% of the remaining pixels for <15% crop expansion.
            # ^^^ very ad-hoc! make this smoother
            remaining_frac = c['sum'] / (total - covered_sum)
            new_area_frac = 1.0 * crop_area(new_crop) / crop_area(crop) - 1
            if new_f1 > f1 or (
                    remaining_frac > 0.25 and new_area_frac < 0.15):
                print('%d %s -> %s / %s (%s), %s -> %s / %s (%s), %s -> %s' % (
                        i, covered_sum, new_sum, total, remaining_frac,
                        crop_area(crop), crop_area(new_crop), area, new_area_frac,
                        f1, new_f1))
                crop = new_crop
                covered_sum = new_sum
                del c_info[i]
                changed = True
                break

        if not changed:
            break

    return crop


def pad_crop(crop, contours, edges, border_contour, pad_px=15):
    """Slightly expand the crop to get full contours.
    This will expand to include any contours it currently intersects, but will
    not expand past a border.
    """
    bx1, by1, bx2, by2 = 0, 0, edges.shape[0], edges.shape[1]
    if border_contour is not None and len(border_contour) > 0:
        c = props_for_contours([border_contour], edges)[0]
        bx1, by1, bx2, by2 = c['x1'] + 5, c['y1'] + 5, c['x2'] - 5, c['y2'] - 5

    def crop_in_border(crop):
        x1, y1, x2, y2 = crop
        x1 = max(x1 - pad_px, bx1)
        y1 = max(y1 - pad_px, by1)
        x2 = min(x2 + pad_px, bx2)
        y2 = min(y2 + pad_px, by2)
        return crop
    
    crop = crop_in_border(crop)

    c_info = props_for_contours(contours, edges)
    changed = False
    for c in c_info:
        this_crop = c['x1'], c['y1'], c['x2'], c['y2']
        this_area = crop_area(this_crop)
        int_area = crop_area(intersect_crops(crop, this_crop))
        new_crop = crop_in_border(union_crops(crop, this_crop))
        if 0 < int_area < this_area and crop != new_crop:
            print( '%s -> %s' % (str(crop), str(new_crop)))
            changed = True
            crop = new_crop

    if changed:
        return pad_crop(crop, contours, edges, border_contour, pad_px)
    else:
        return crop


def downscale_image(im, max_dim=2048):
    """Shrink im until its longest dimension is <= max_dim.
    Returns new_image, scale (where scale <= 1).
    """
    a, b = im.size
    if max(a, b) <= max_dim:
        return 1.0, im

    scale = 1.0 * max_dim / max(a, b)
    new_im = im.resize((int(a * scale), int(b * scale)), Image.ANTIALIAS)
    return scale, new_im

In [33]:
def set_image_dpi(im):
    length_x, width_y = im.size
    factor = min(1, float(1024.0 / length_x))
    size = int(factor * length_x), int(factor * width_y)
    im_resized = im.resize(size, Image.ANTIALIAS)
    return im_resized
    #temp_file = tempfile.NamedTemporaryFile(delete=False,   suffix='.png')
    #temp_filename = temp_file.name
    #im_resized.save(temp_filename, dpi=(300, 300))
    #return temp_filename

In [54]:
#def process_image(path, out_path):
path = "test.jpg"
out_path= "test.crop.jpg"

orig_im = Image.open(path)
#orig_im = set_image_dpi(orig_im)
scale, im = downscale_image(orig_im)
#convolve the image with gaussian blur for Canny
#im_blured = cv2.GaussianBlur(np.asarray(im), (5, 5), 0)
edges = cv2.Canny(im_blured, 100, 200)

# TODO: dilate image _before_ finding a border. This is crazy sensitive!
_, contours, hierarchy = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
borders = find_border_components(contours, edges)
print("OK")
borders.sort(key=lambda i, x1, y1, x2, y2: (x2 - x1) * (y2 - y1))
print("OK")
border_contour = None
if len(borders):
    border_contour = contours[borders[0][0]]
    edges = remove_border(border_contour, edges)

edges = 255 * (edges > 0).astype(np.uint8)

# Remove ~1px borders using a rank filter.
maxed_rows = rank_filter(edges, -4, size=(1, 20))
maxed_cols = rank_filter(edges, -4, size=(20, 1))
debordered = np.minimum(np.minimum(edges, maxed_rows), maxed_cols)
edges = debordered.astype(np.int32)
print(edges.shape)

contours = find_components(edges)
if len(contours) == 0:
    print('%s -> (no text!)' % path)
    #return

crop = find_optimal_components_subset(contours, edges)
crop = pad_crop(crop, contours, edges, border_contour)

crop = [int(x / scale) for x in crop]  # upscale to the original image size.
#show borders
draw = ImageDraw.Draw(im)
c_info = props_for_contours(contours, edges)
"""
for c in c_info:
    this_crop = c['x1'], c['y1'], c['x2'], c['y2']
    draw.rectangle(this_crop, outline='blue')
draw.rectangle(crop, outline='red')
im.save(out_path)
draw.text((50, 50), path, fill='red')
orig_im.save(out_path)
im.show()
text_im = orig_im.crop(crop)
text_im.save(out_path)
print('%s -> %s' % (path, out_path))
"""


FileNotFoundError: [Errno 2] No such file or directory: 'test.jpg'

In [53]:
i = 0
croped_im.crop((c_info[i]['x1'], c_info[i]['y1'],c_info[i]['x2'], c_info[i]['y2'] ))

In [50]:
croped_im.save(out_path)